# Lab 7 - Event Selection Optimalization

##### PHYS434 - Advanced Laboratory: Computational Data Analysis
##### Professor: Miguel Morales
<br>

##### Due date: 12/6/2021
##### By Erik Solhaug
<br>

In this lab, we will explore our data and apply confidence intervals.

From the previous lab, we have the data presented below.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import h5py
import pickle
import math

import pandas as pd

#this sets the size of the plot to something useful
plt.rcParams["figure.figsize"] = (20,15)

In [2]:
user_dir = '/home/jovyan/Phys434_Autumn2021/'
data_dir = user_dir + 'LHC_data/'
data_dir

'/home/jovyan/Phys434_Autumn2021/LHC_data/'

In [3]:
with open(f'{data_dir}/higgs_100000_pt_250_500.pkl', 'rb') as f:
    higgs = pickle.load(f)

In [4]:
with open(f'{data_dir}/qcd_100000_pt_250_500.pkl', 'rb') as f:
    qcd = pickle.load(f)